In [1]:
import lsst.sims.maf
lsst.sims.maf.__version__

'2.6.0.sims'

In [2]:
%matplotlib inline   
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

## Run minion_1016

In [4]:
# setup database connection
outDir ='Cadence_compare'
Baseline = '/home/mount/Opsim DB/minion_1016_sqlite.db'
RollingV2 = '/home/mount/Opsim DB/enigma_1260_sqlite.db'
BL_noPair = '/home/mount/Opsim DB/kraken_1043_sqlite.db'

# specify output directory for metrics result
resultsDb = db.ResultsDb(outDir=outDir)

In [5]:
from numpy import amin as m

In [6]:
# retrive min inter_night gap, and observation history with the input of database file name and
# arrays of RA and DEC
def find_min_gap(dbFile, ra, dec):
    # establish connection to sqllite database file
    opsimdb = db.OpsimDatabase(dbFile)
    
    # While we're in transition between opsim v3 and v4, this may be helpful:
#   print("{dbFile} is an opsim version {version} database".format(dbFile=dbFile, version=opsimdb.opsimVersion))
    if opsimdb.opsimVersion == "V3":
        # For v3 databases:
        mjdcol = 'expMJD'
        degrees = False
    else:
        # For v4 and alternate scheduler databases.
        mjdcol = 'observationStartMJD'
        degrees = True
    
    # IntraNightGapsMetric returns the gap (in days) between observations within the same night
    # custom reduceFunc to find min gaps 
    metric = metrics.cadenceMetrics.IntraNightGapsMetric(reduceFunc=np.amin, mjdCol=mjdcol)
    # PassMetric just pass all values
    metric_pass = metrics.simpleMetrics.PassMetric(cols=['filter','fiveSigmaDepth', mjdcol, 'expDate'])
    # slicer for slicing pointing history
    slicer = slicers.UserPointsSlicer(ra, dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg=degrees)
    # sql constrains, here I put none
    sql = '' #'night < 365'
    
    # bundles to combine metric, slicer and sql constrain together
    bundle = metricBundles.MetricBundle(metric,slicer,sql)
    date_bundle = metricBundles.MetricBundle(metric_pass, slicer, sql)
    # In case you are using a dither stacker, we can check what columns are actually being pulled from the database.
    print(bundle.dbCols)
    
    # create metric bundle group and returns
    bg = metricBundles.MetricBundleGroup({'sep': bundle, 'cadence':date_bundle}, opsimdb, outDir=outDir, resultsDb=resultsDb)
    bg.runAll()
    opsimdb.close()
    return bg

In [7]:
ra = 20
dec = 0

In [8]:
# provide the ra, dec (must be in degrees) and the sql_lite database dir
# 1st is the baseline cadence
NP_result = find_min_gap(BL_noPair, ra, dec)

{'night', 'expMJD', 'fieldRA', 'fieldDec'}
Querying database Summary with no constraint for columns ['expDate', 'expMJD', 'fieldRA', 'night', 'filter', 'fiveSigmaDepth', 'fieldDec'].
Found 2509335 visits
Running:  ['sep', 'cadence']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [9]:
# Now baseline with no pairs
BL_result = find_min_gap(Baseline, ra, dec)

{'night', 'expMJD', 'fieldRA', 'fieldDec'}
Querying database Summary with no constraint for columns ['expDate', 'expMJD', 'fieldRA', 'night', 'filter', 'fiveSigmaDepth', 'fieldDec'].
Found 2447931 visits
Running:  ['sep', 'cadence']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [10]:
# Now baseline with no pairs
R_result = find_min_gap(RollingV2, ra, dec)

{'night', 'expMJD', 'fieldRA', 'fieldDec'}
Querying database Summary with no constraint for columns ['expDate', 'expMJD', 'fieldRA', 'night', 'filter', 'fiveSigmaDepth', 'fieldDec'].
Found 2284261 visits
Running:  ['sep', 'cadence']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [11]:
# assign min intra_night gap in hours to gap
gap1 = BL_result.bundleDict['sep'].metricValues.data[0]
# assign pointing history to obsHist
cadence1 = BL_result.bundleDict['cadence'].metricValues.data[0]
# put ra, dec, and gap into array 'pos_gap'
meta1 = [ra, dec, gap1, 'minion_1016']

# Baseline no pairs
gap2 = NP_result.bundleDict['sep'].metricValues.data[0]
cadence2 = NP_result.bundleDict['cadence'].metricValues.data[0]
meta2 = [ra, dec, gap2,'kraken_1043']

# Rolling cadence
gap3 = R_result.bundleDict['sep'].metricValues.data[0]
cadence3 = R_result.bundleDict['cadence'].metricValues.data[0]
meta3= [ra, dec, gap3, 'enigma_1260']

# Now let's save it to a file, .npz will be automatically added
outfile = '/home/mount/MAF output/all_three'
np.savez(outfile, meta1=meta1, cadence1=cadence1, meta2=meta2, cadence2=cadence2, meta3=meta3, cadence3=cadence3)